In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Liver_cirrhosis"
cohort = "GSE182060"

# Input paths
in_trait_dir = "../../input/GEO/Liver_cirrhosis"
in_cohort_dir = "../../input/GEO/Liver_cirrhosis/GSE182060"

# Output paths
out_data_file = "../../output/preprocess/Liver_cirrhosis/GSE182060.csv"
out_gene_data_file = "../../output/preprocess/Liver_cirrhosis/gene_data/GSE182060.csv"
out_clinical_data_file = "../../output/preprocess/Liver_cirrhosis/clinical_data/GSE182060.csv"
json_path = "../../output/preprocess/Liver_cirrhosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Prognostic liver signature profiles in biopsy tissues from non-alcoholic fatty liver disease patients followed for fibrosis progression"
!Series_summary	"Background/Aims: There is a major unmet need to assess prognostic impact of anti-fibrotics in clinical trials due to the slow rate of liver fibrosis progression. We aimed to develop a surrogate biomarker to predict future fibrosis progression."
!Series_summary	"Methods: A Fibrosis Progression Signature (FPS) was defined to predict fibrosis progression within 5 years in HCV and NAFLD patients with no to minimal fibrosis at baseline (n=421), and validated in an independent NAFLD cohort (n=78). The FPS was used to assess response to 13 candidate anti-fibrotics in organotypic ex vivo cultures of clinical fibrotic liver tissues (n=78), and cenicriviroc in NASH patients enrolled in a clinical trial (n=19, NCT02217475). A serum-protein-based surrogate FPS (FPSec) was developed and technically evaluated 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
import numpy as np
from typing import Optional, Callable, Dict, Any, List, Union

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from liver biopsy tissues
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the Sample Characteristics Dictionary:
# 0: Patient IDs - not a trait
# 1: tissue type - not a trait, it's constant "liver biopsy"
# 2: time_point - this can be used to identify cirrhosis progression

# 2.1 Data Availability
trait_row = 2  # time_point can be used to identify cirrhosis progression
age_row = None  # Age information is not available in the sample characteristics
gender_row = None  # Gender information is not available in the sample characteristics

# 2.2 Data Type Conversion
def convert_trait(value: str) -> Optional[int]:
    """Convert time_point information to binary trait data.
    Baseline (not progressed) = 0, Follow-up (progressed) = 1
    """
    if value is None or pd.isna(value):
        return None
    
    # Extract the value if it contains a colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'Baseline' in value:
        return 0  # Not progressed
    elif 'Follow-up' in value:
        return 1  # Progressed
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """Convert age to continuous value."""
    # This function is defined but won't be used as age data is not available
    if value is None or pd.isna(value):
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary (female=0, male=1)."""
    # This function is defined but won't be used as gender data is not available
    if value is None or pd.isna(value):
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if value in ['female', 'f']:
        return 0
    elif value in ['male', 'm']:
        return 1
    else:
        return None

# 3. Save Metadata
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create clinical_data DataFrame from the sample characteristics dictionary provided in the previous step
    sample_characteristics = {
        0: ['patient: HUc034', 'patient: HUc035', 'patient: HUc036', 'patient: HUc037', 'patient: HUc038', 
            'patient: HUc039', 'patient: HUc041', 'patient: HUc042', 'patient: HUc043', 'patient: HUc044', 
            'patient: HUc045', 'patient: HUc046', 'patient: HUc047', 'patient: HUc048', 'patient: HUc049', 
            'patient: HUc050', 'patient: HUc051', 'patient: HUc052', 'patient: HUc053', 'patient: HUc054', 
            'patient: HUc055', 'patient: HUc056', 'patient: HUc057', 'patient: HUc058', 'patient: HUc059', 
            'patient: HUc060', 'patient: HUc061', 'patient: HUc062', 'patient: HUc063', 'patient: HUc064'],
        1: ['tissue: liver biopsy'] * 30,  # Repeating the same value for all 30 patients
        2: ['time_point: Baseline', 'time_point: Follow-up'] * 15  # Alternating pattern to match the 30 patients
    }
    
    # Convert to DataFrame
    clinical_data = pd.DataFrame(sample_characteristics)
    
    # Extract clinical features using the function from the library
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of clinical data:")
    print(preview)
    
    # Ensure directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{0: [nan], 1: [nan], 2: [0.0]}
Clinical data saved to ../../output/preprocess/Liver_cirrhosis/clinical_data/GSE182060.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Liver_cirrhosis/GSE182060/GSE182060_series_matrix.txt.gz
Gene data shape: (202, 156)
First 20 gene/probe identifiers:
Index(['AARS', 'ABLIM1', 'ACOT2', 'ACSM3', 'ACTR2', 'ADD3', 'ADH5', 'ADH6',
       'ADRA2B', 'AEBP1', 'AKAP13', 'AKR1A1', 'AKR1D1', 'ALAS1', 'ALDH9A1',
       'ANKRD46', 'ANXA1', 'ANXA3', 'AOX1', 'AP1B1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These appear to be standard human gene symbols (like AARS, ABLIM1, etc.)
# They follow the standard gene symbol nomenclature and are recognizable human genes

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to liver fibrosis progression, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data for liver fibrosis progression, which is relevant to liver cirrhosis research."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for liver cirrhosis analysis."
    )
    
    print("Dataset is not usable for liver cirrhosis analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (202, 156)
Gene data shape after normalization: (201, 156)
Normalized gene expression data saved to ../../output/preprocess/Liver_cirrhosis/gene_data/GSE182060.csv
Original clinical data preview:
         !Sample_geo_accession            GSM5517043            GSM5517044  \
0  !Sample_characteristics_ch1       patient: HUc034       patient: HUc035   
1  !Sample_characteristics_ch1  tissue: liver biopsy  tissue: liver biopsy   
2  !Sample_characteristics_ch1  time_point: Baseline  time_point: Baseline   

             GSM5517045            GSM5517046            GSM5517047  \
0       patient: HUc036       patient: HUc037       patient: HUc038   
1  tissue: liver biopsy  tissue: liver biopsy  tissue: liver biopsy   
2  time_point: Baseline  time_point: Baseline  time_point: Baseline   

             GSM5517048            GSM5517049            GSM5517050  \
0       patient: HUc039       patient: HUc041       patient: HUc042   
1  tissue: liver biopsy  t